# Prompt + LLM

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

## PromptTemplate + LLM

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

In [3]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears like fast food? Because they can't bear the wait!")

### Attaching Stop Sequences

In [4]:
chain = prompt | model.bind(stop=["\n"])

In [5]:
chain.invoke({"foo": "bears"})

AIMessage(content='Why did the bear break up with his girlfriend? ')

### Attaching Function Call information

In [6]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [7]:
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"Why don\'t bears like fast food?","punchline":"Because they can\'t catch it!"}', 'name': 'joke'}})

## PromptTemplate + LLM + OutputParser

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [9]:
chain.invoke({"foo": "bears"})

'Why do bears have hairy coats?\n\nFur protection!'

### Functions Output Parser

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [11]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

In [12]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [13]:
chain.invoke({"foo": "bears"})

"Why don't bears like fast food?"

## Simplifying input

In [14]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [15]:
chain.invoke("bears")

'Why do bears have hairy coats?'

In [16]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [17]:
chain.invoke("bears")

"Why don't bears like fast food?"